In [24]:
import numpy as np
from typing import Tuple

## Functions


In [2]:
def score(x_1: np.ndarray, x_2: np.ndarray) -> int:
    """
    Compares two 1D NumPy arrays and returns a score.

    Parameters:
      x_1: The first 1D NumPy array.
      x_2: The second 1D NumPy array.

    Returns:
      The score, which is the number of elements in x_1 that are greater than or equal to the corresponding elements in x_2.
    """

    score = 0
    for i in range(x_1.shape[0]):
        score += np.sum(x_1[i] >= x_2)

    return score

## step 1


In [33]:
n, p = (100, 100)
X = np.random.randn(n, p)
y = np.random.randint(0, 2, size=n)
print(X.shape)
print(np.unique(y, return_counts=True))

(100, 100)
(array([0, 1]), array([46, 54], dtype=int64))


## step 2


In [34]:
_, (n0, n1) = np.unique(y, return_counts=True)

s_max = n0 * n1  # Maximum possible score
s_min = 0
# Step 2: Sign-flip operation
scores = np.zeros(p)
new_scores = np.zeros(p)
for i in range(p):
    xi = X[:, i]
    class_0_values, class_1_values = xi[y == 0], xi[y == 1]
    scores[i] = score(class_0_values, class_1_values)
    if scores[i] > s_max / 2:
        X[:, i] *= -1
    new_scores[i] = min(scores[i], s_max - scores[i])
X.shape, new_scores.shape, s_max

((100, 100), (100,), 2484)

## step 3


In [35]:
cluster = []
cluster_index = []
margins = np.min(X[y == 1], axis=0) - np.max(X[y == 0], axis=0)

if len(cluster) == 0:
    min_value = np.min(new_scores)
    min_indices = np.where(new_scores == min_value)[0]

    i_star = (
        np.argmin(new_scores)
        if min_indices.size == 1
        else np.argmax(margins[min_indices])
    )
    cluster = np.expand_dims(X[:, i_star], axis=1)
    cluster_index.append(i_star)
    initial_cluster_mean = X[:, i_star]
else:
    # Random selected genes
    pass

cluster.shape, cluster_index,i_star

((100, 1), [57], 57)

In [36]:
initial_cluster_mean

array([ 0.43774854, -0.70269719, -0.65516156,  2.51799226, -1.04708448,
       -0.24788164, -1.16590494, -1.74005918,  0.59843775,  0.56796723,
        1.20460126,  0.08429109,  0.6581176 , -0.66757093, -0.84927058,
       -0.76463869, -0.13021505,  0.16778205,  1.28113971,  2.46951282,
        1.11935643, -0.18076851, -0.83555809,  0.60633124,  0.1125101 ,
       -0.60923126,  0.51129733, -2.4765508 , -0.23683659, -0.56054785,
        2.27466315, -0.02027489, -0.3363442 ,  1.39916693,  1.23731564,
       -0.58639245, -1.13772259,  0.74260974,  2.3783406 ,  0.55108076,
        1.53007435,  0.39873528, -0.17470299,  2.01039721, -0.94090366,
        0.79705562, -1.57736213, -0.32694543,  0.29892763, -1.72275012,
        0.41572058,  1.64395338, -0.68947232, -0.85807489,  0.71771444,
        0.17486836,  0.03893379, -1.48774675, -0.30322877,  0.64523587,
       -1.17022065,  0.55487068,  0.02536371, -0.26818659, -0.64763202,
       -0.8355553 ,  0.27499694, -0.05820931, -0.67774144,  0.49

## step 4-5

In [13]:
# forward search
cluster_mean_score = score(initial_cluster_mean[y == 0], initial_cluster_mean[y == 1])
cluster_mean_margin = np.min(initial_cluster_mean[y == 1], axis=0) - np.max(
    initial_cluster_mean[y == 0], axis=0
)

while True:
    scores_with_gene = np.zeros(p)
    margins_with_gene = np.zeros(p)
    for i in range(p):
        if i in cluster_index:
            scores_with_gene[i] = s_max + 1
            margins_with_gene[i] = 0
            continue
        temp_cluster = np.concatenate(
            [cluster, np.expand_dims(X[:, i], axis=1)], axis=1
        )
        temp_cluster_avg = np.mean(temp_cluster, axis=1)
        scores_with_gene[i] = score(temp_cluster_avg[y == 0], temp_cluster_avg[y == 1])
        margins_with_gene[i] = np.min(temp_cluster_avg[y == 1]) - np.max(
            temp_cluster_avg[y == 0]
        )

    min_value = np.min(scores_with_gene)
    min_indices = np.where(scores_with_gene == min_value)[0]

    i_star = (
        min_indices[0]
        if min_indices.size == 1
        else np.argmax(margins_with_gene[min_indices])
    )

    if scores_with_gene[i_star] >= cluster_mean_score:
        break

    if scores_with_gene[i_star] == cluster_mean_score:
        if margins_with_gene[i_star] <= cluster_mean_margin:
            break

    cluster_index.append(i_star)
    cluster = np.concatenate([cluster, np.expand_dims(X[:, i_star], axis=1)], axis=1)
    cluster_mean_score = scores_with_gene[i_star]
    cluster_mean_margin = margins_with_gene[i_star]

In [14]:
cluster.shape, cluster_index

((100, 9), [42, 75, 88, 77, 38, 56, 74, 85, 95])

## step 6-7

In [15]:
# Backward search

initial_cluster_mean = np.mean(cluster, axis=1)
cluster_mean_score = score(initial_cluster_mean[y == 0], initial_cluster_mean[y == 1])
cluster_mean_margin = np.min(initial_cluster_mean[y == 1], axis=0) - np.max(
    initial_cluster_mean[y == 0], axis=0
)

while True:
    scores_without_gene = np.zeros(len(cluster_index))
    margins_without_gene = np.zeros(len(cluster_index))
    # start from last gene
    for i in range(-1, -len(cluster_index) - 1, -1):
        cluster_mean_without_gene_i = np.mean(np.delete(cluster, i, axis=1), axis=1)
        scores_without_gene[i] = score(
            cluster_mean_without_gene_i[y == 0], cluster_mean_without_gene_i[y == 1]
        )
        margins_without_gene[i] = np.min(
            cluster_mean_without_gene_i[y == 1], axis=0
        ) - np.max(cluster_mean_without_gene_i[y == 0], axis=0)

    min_value = np.min(scores_without_gene)
    min_indices = np.where(scores_without_gene == min_value)[0]

    i_star = (
        min_indices[0]
        if min_indices.size == 1
        else np.argmax(margins_without_gene[min_indices])
    )
    if scores_without_gene[i_star] > cluster_mean_score:
        break

    if scores_without_gene[i_star] == cluster_mean_score:
        if margins_without_gene[i_star] <= cluster_mean_margin:
            break

    cluster = np.delete(cluster, i_star, axis=1)
    cluster_index = np.delete(cluster_index, i_star, axis=0)
    cluster_mean_score = scores_without_gene[i_star]
    cluster_mean_margin = margins_without_gene[i_star]
initial_cluster_mean=np.mean(cluster, axis=1)

In [22]:
 cluster_index

[42, 75, 88, 77, 38, 56, 74, 85, 95]

## step 8 (4-7)
To excute this step excute only 1,2,3 steps.

In [37]:
def forward_search(
    X: np.ndarray,
    y: np.ndarray,
    cluster: np.ndarray,
    cluster_index: np.ndarray,
    initial_cluster_mean: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Forward Search (step 4-5).

    Parameters:
        X (2D NumPy array): The data array of shape (num_samples, num_genes).
        y (1D NumPy array): The label array of shape (num_samples,).
        cluster (2D NumPy array): The selected gene or genns.
        cluster_index (1D NumPy array): The index of selected gene or genns.
        initial_cluster_mean (1D NumPy array): The mean of cluster.

    Returns:
        cluster: A 2D NumPy array of shape (num_samples, num_genes) containing the clustered genes.
        cluster_index: A 1D NumPy array of shape (num_genes,) containing the index of clustered genes.
    """

    n, p = X.shape
    _, (n0, n1) = np.unique(y, return_counts=True)

    s_max = n0 * n1  # Maximum possible score
    cluster_mean_score = score(
        initial_cluster_mean[y == 0], initial_cluster_mean[y == 1]
    )
    cluster_mean_margin = np.min(initial_cluster_mean[y == 1], axis=0) - np.max(
        initial_cluster_mean[y == 0], axis=0
    )

    while True:
        scores_with_gene = np.zeros(p)
        margins_with_gene = np.zeros(p)
        for i in range(p):
            if i in cluster_index:
                # s_max+1 is the big score for to not be selected as i_star
                scores_with_gene[i] = s_max + 1
                # impossible to found gene with margin 0 for to not be selected as i_star
                margins_with_gene[i] = 0
                continue
            temp_cluster = np.concatenate(
                [cluster, np.expand_dims(X[:, i], axis=1)], axis=1
            )
            temp_cluster_avg = np.mean(temp_cluster, axis=1)
            scores_with_gene[i] = score(
                temp_cluster_avg[y == 0], temp_cluster_avg[y == 1]
            )
            margins_with_gene[i] = np.min(temp_cluster_avg[y == 1]) - np.max(
                temp_cluster_avg[y == 0]
            )

        min_value = np.min(scores_with_gene)
        min_indices = np.where(scores_with_gene == min_value)[0]

        i_star = (
            min_indices[0]
            if min_indices.size == 1
            else np.argmax(margins_with_gene[min_indices])
        )

        if scores_with_gene[i_star] >= cluster_mean_score:
            break

        if scores_with_gene[i_star] == cluster_mean_score:
            if margins_with_gene[i_star] <= cluster_mean_margin:
                break

        cluster_index = np.append(cluster_index, i_star)
        cluster = np.concatenate(
            [cluster, np.expand_dims(X[:, i_star], axis=1)], axis=1
        )
        cluster_mean_score = scores_with_gene[i_star]
        cluster_mean_margin = margins_with_gene[i_star]
    return cluster, cluster_index


def backword_search(
    y: np.ndarray,
    cluster: np.ndarray,
    cluster_index: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Backword Search (step 6-7).

    Parameters:
        y (1D NumPy array): The label array of shape (num_samples,).
        cluster (2D NumPy array): The selected gene or genns.
        cluster_index (1D NumPy array): The index of selected gene or genns.

    Returns:
        cluster: A 2D NumPy array of shape (num_samples, num_genes) containing the clustered genes.
        cluster_index: A 1D NumPy array of shape (num_genes,) containing the index of clustered genes.
        initial_cluster_mean: A 1D NumPy array of shape (num_samples,) containing the mean of cluster.
    """

    initial_cluster_mean = np.mean(cluster, axis=1)
    cluster_mean_score = score(
        initial_cluster_mean[y == 0], initial_cluster_mean[y == 1]
    )
    cluster_mean_margin = np.min(initial_cluster_mean[y == 1], axis=0) - np.max(
        initial_cluster_mean[y == 0], axis=0
    )

    while True:
        scores_without_gene = np.zeros(cluster_index.size)
        margins_without_gene = np.zeros(cluster_index.size)
        # start from last gene
        for i in range(-1, -cluster_index.size - 1, -1):
            cluster_mean_without_gene_i = np.mean(np.delete(cluster, i, axis=1), axis=1)
            scores_without_gene[i] = score(
                cluster_mean_without_gene_i[y == 0], cluster_mean_without_gene_i[y == 1]
            )
            margins_without_gene[i] = np.min(
                cluster_mean_without_gene_i[y == 1], axis=0
            ) - np.max(cluster_mean_without_gene_i[y == 0], axis=0)

        min_value = np.min(scores_without_gene)
        min_indices = np.where(scores_without_gene == min_value)[0]

        i_star = (
            min_indices[0]
            if min_indices.size == 1
            else np.argmax(margins_without_gene[min_indices])
        )
        if scores_without_gene[i_star] > cluster_mean_score:
            break

        if scores_without_gene[i_star] == cluster_mean_score:
            if margins_without_gene[i_star] <= cluster_mean_margin:
                break

        cluster = np.delete(cluster, i_star, axis=1)
        cluster_index = np.delete(cluster_index, i_star, axis=0)
        cluster_mean_score = scores_without_gene[i_star]
        cluster_mean_margin = margins_without_gene[i_star]
    initial_cluster_mean = np.mean(cluster, axis=1)
    return cluster, cluster_index, initial_cluster_mean

In [38]:
cluster, cluster_index = forward_search(
    X, y, cluster, cluster_index, initial_cluster_mean
)
cluster, cluster_index, intiale_cluster_mean = backword_search(
    y, cluster, cluster_index
)
cluster_index_state = []
cluster_index_state.append(cluster_index)
while True:
    cluster, cluster_index = forward_search(
        X, y, cluster, cluster_index, intiale_cluster_mean
    )
    cluster, cluster_index, intiale_cluster_mean = backword_search(
        y, cluster, cluster_index
    )
    cluster_index_state.append(cluster_index)

    if np.array_equal(cluster_index_state[-2], cluster_index_state[-1]):
        break

[array([57, 33, 31, 10, 18, 22, 40, 12,  0, 69, 16, 48, 81, 24, 75],
      dtype=int64)]
[array([57, 33, 31, 10, 18, 22, 40, 12,  0, 69, 16, 48, 81, 24, 75],
      dtype=int64), array([57, 33, 31, 10, 18, 22, 40, 12,  0, 69, 16, 48, 81, 24, 75],
      dtype=int64)]


In [39]:
cluster_index

array([57, 33, 31, 10, 18, 22, 40, 12,  0, 69, 16, 48, 81, 24, 75],
      dtype=int64)

## for the step 9 run main.py